In [1]:
# Our main plotting package (must have explicit import of submodules)
import bokeh.io
import bokeh.plotting
import sys
import glob 
import os 
import scipy.io as sio
import numpy as np
import decimal as dc

from bokeh.models import Legend

from pathlib import Path

from scipy.stats import norm

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

Loading BokehJS ...

In [11]:
mouseName = 'GF175'

WhMissInaRow = 50

behavpath = '\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\BehaviourFiles\\'

imaging_days = glob.glob(os.path.join(behavpath, "GF*"))

for n, f in enumerate(imaging_days):
    imaging_days[n] = os.path.basename(f)

for n, d in enumerate(imaging_days):
    
    behav_path = behavpath + d + '\\BehavResults.mat'
    
    print(behav_path)
    
    beh = sio.loadmat(behav_path)

    vals = beh['BehavResults'][0,0]

    keys = beh['BehavResults'][0,0].dtype.descr

    beh = np.array(vals[keys[0][0]][:][:])

    performance = np.array(beh[:,9])
    performance = performance.astype(int)
    stim = np.array(beh[:,5])
    stim = stim.astype(int)
    whstim = np.array(beh[:,6])
    whstim = whstim.astype(int)
    audstim =  np.array(beh[:,7])
    audstim = audstim.astype(int)
    
    
    
    AudHit = 0
    AudMiss = 0
    WhHit = 0
    WhMiss = 0
    FA = 0
    CR = 0
    MissFlag = []
    WhStim = 0
    AudStim = 0
    NoStim = 0
    StopSession = False
    TotalTrials = 0
    TrialNumber = len(beh)
    AudHitRate = []
    WhHitRate = []
    FARate = []
    RecentTrials = 20
    WhHitChunk = np.zeros(TrialNumber)
    AudHitChunk = np.zeros(TrialNumber)
    FAChunk = np.zeros(TrialNumber)
    dprime = np.zeros(TrialNumber)
    
    print('Number of Trials ' + str(TrialNumber))
    
    for n in range(TrialNumber):
        
        ChunkPer = performance[np.max([0,n-RecentTrials]) : (n+1)]
        ChunkStim = stim[np.max([0,n-RecentTrials]) : (n+1)]
        ChunkWhStim = whstim[np.max([0,n-RecentTrials]) : (n+1)]
        ChunkAuStim = audstim[np.max([0,n-RecentTrials]) : (n+1)]
        WhHitChunk[n] = sum((ChunkPer == 2))/sum((ChunkWhStim == 1))        
        AudHitChunk[n] = sum((ChunkPer == 3))/sum((ChunkAuStim == 1))
        FAChunk[n] = sum((ChunkPer == 5))/sum((ChunkStim == 0))
        Half_WhHit = 0.5/(sum((ChunkPer == 2)) + sum((ChunkPer == 0)))
        Half_FA = 0.5/(sum((ChunkPer == 5)) + sum((ChunkPer == 4)) ) 
        
        if WhHitChunk[n] == 1:
            
            WHit = 1-Half_WhHit
            
        elif WhHitChunk[n] == 0:
            
            WHit = Half_WhHit
        else:
            WHit = WhHitChunk[n]
            
        if FAChunk[n] == 1:
            
            FA = 1-Half_FA
            
        elif FAChunk[n] == 0:
            
            FA = Half_FA
            
        else:
            FA = FAChunk[n]
            
        dprime[n] = norm.ppf(WHit) - norm.ppf(FA)
        
        
    print(TrialNumber)
    print(len(WhHitChunk))

    p = bokeh.plotting.figure(
    width=800,
    height=500,
    x_axis_label='Trial Number',
    y_axis_label='AudHit/WhHit/FA',
    x_range=(1, TrialNumber),
    title= mouseName + '|' + d 
    
    )
    

    p.yaxis.axis_label_text_font_style = "bold"
    p.xaxis.axis_label_text_font_style = "bold"
    p.title.align = 'center'
    

    audh = p.line(
    x= range(TrialNumber),
    y= np.array(AudHitChunk),
    line_join='bevel',
    line_width=2,
    color = 'blue',
    )
    
    fa = p.line(
    x= range(TrialNumber),
    y= np.array(FAChunk),
    line_join='bevel',
    line_width=2,
    color = 'red',
    )
    
       
    whhit = p.line(
    x= range(TrialNumber),
    y= np.array(WhHitChunk),
    line_join='bevel',
    line_width=2,
    color = 'black',
    )
    
    d = p.line(
    x= range(TrialNumber),
    y= np.array(dprime),
    line_join='bevel',
    line_width=2,
    color = 'green',
    )
    
    legend = Legend(items=[
    ("AudHit" , [audh]),
    ("WhHit" , [whhit]),
    ("FA" , [fa]),
    ("d'", [d])   
    ], location="center")
    
    p.add_layout(legend, 'right')
    
    p.legend.click_policy="hide"

    bokeh.io.show(p)
    
    for k in range(TrialNumber):
        
        if not(StopSession):
            
            TotalTrials += 1

            if performance[k] == 0:

                WhMiss += 1 

                if len(MissFlag) < WhMissInaRow:

                    MissFlag.append(1)
                else:

                    MissFlag = MissFlag[1::]
                    MissFlag.append(1)

                if (sum(MissFlag) == WhMissInaRow):

                    print(str(WhMissInaRow) + 'WhMiss in row detected')

                    StopSession = True

            elif performance[k] == 1:

                AudMiss += 1

                if len(MissFlag) < WhMissInaRow:

                    MissFlag.append(0)
                else:

                    MissFlag = MissFlag[1::]
                    MissFlag.append(0)

                if (sum(MissFlag) == WhMissInaRow):

                    print(str(WhMissInaRow) + 'WhMiss in row detected')

                    StopSession = True

            elif performance[k] == 2:

                WhHit += 1

                if len(MissFlag) < WhMissInaRow:

                    MissFlag.append(0)
                else:

                    MissFlag = MissFlag[1::]
                    MissFlag.append(0)

                if (sum(MissFlag) == WhMissInaRow):

                    print(str(WhMissInaRow) + 'WhMiss in row detected')

                    StopSession = True

            elif performance[k] == 3:

                AudHit += 1

                if len(MissFlag) < WhMissInaRow:

                    MissFlag.append(0)
                else:

                    MissFlag = MissFlag[1::]
                    MissFlag.append(0)

                if (sum(MissFlag) == WhMissInaRow):

                    print(str(WhMissInaRow) + 'WhMiss in row detected')

                    StopSession = True

            elif performance[k] == 4:

                CR += 1

                if len(MissFlag) < WhMissInaRow:

                    MissFlag.append(0)
                else:

                    MissFlag = MissFlag[1::]
                    MissFlag.append(0)

                if (sum(MissFlag) == WhMissInaRow):

                    print(str(WhMissInaRow) + 'WhMiss in row detected')

                    StopSession = True

            elif performance[k] == 5:

                FA += 1


                if len(MissFlag) < WhMissInaRow:

                    MissFlag.append(0)
                else:

                    MissFlag = MissFlag[1::]
                    MissFlag.append(0)

                if (sum(MissFlag) == WhMissInaRow):
                
                    print(str(WhMissInaRow) + 'WhMiss in row detected')

                    StopSession = True
                    
            
                    
            if whstim[k] == 1:
                
                WhStim += 1
                
            if audstim[k] == 1:
                
                AudStim += 1
                
            if  stim[k] == 0:
                
                NoStim += 1
                
            
    AudHitRate.append(AudHit/AudStim)
    
    print(AudHitRate)
            
    WhHitRate.append(WhHit/WhStim)
            
    FARate.append(FA/NoStim)
            
        
print(WhHitRate) 

        


\\sv1files.epfl.ch\Petersen-Lab\data\GF175\Recordings\BehaviourFiles\GF175_14012019\BehavResults.mat
Number of Trials 631


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in long_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in long_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in long_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in true_divide


631
631


[0.8819444444444444]
\\sv1files.epfl.ch\Petersen-Lab\data\GF175\Recordings\BehaviourFiles\GF175_15012019\BehavResults.mat
Number of Trials 650
650
650


[0.8125]
\\sv1files.epfl.ch\Petersen-Lab\data\GF175\Recordings\BehaviourFiles\GF175_17012019\BehavResults.mat
Number of Trials 552
552
552


[0.9197530864197531]
\\sv1files.epfl.ch\Petersen-Lab\data\GF175\Recordings\BehaviourFiles\GF175_18012019\BehavResults.mat
Number of Trials 1007
1007
1007


[0.9743589743589743]
\\sv1files.epfl.ch\Petersen-Lab\data\GF175\Recordings\BehaviourFiles\GF175_21012019\BehavResults.mat
Number of Trials 643
643
643


[0.9629629629629629]
\\sv1files.epfl.ch\Petersen-Lab\data\GF175\Recordings\BehaviourFiles\GF175_22012019\BehavResults.mat
Number of Trials 609
609
609


[0.9803921568627451]
[0.6483516483516484]


In [13]:
norm.ppf(WhHitChunk[0])

inf

In [29]:
dprime

array([        nan,         nan,         nan,         nan,         nan,
        0.67448975,  0.96742157,  1.64191132,  1.64191132,  1.93484313,
        1.93484313,  1.93484313,  2.11777095,  2.11777095,  2.11777095,
        2.30069876,  2.43190095,  2.43190095,  2.53334351,  2.66454569,
        2.66454569,  2.66454569,  2.66454569,  2.74678536,  2.74678536,
        2.84822792,  2.84822792,  2.84822792,  2.53334351,  2.60363448,
        2.68587414,  2.68587414,  2.74678536,  2.81567211,  2.81567211,
        2.44257841,  2.50167669,  2.50167669,  2.56064038,  2.61227519,
        2.66492986,  2.66492986,  2.66492986,  2.71242415,  2.71242415,
        2.75819215,  2.53224286,  2.53224286,  2.57776819,  2.57776819,
        2.61881096,  2.57776819,  2.53200019,  2.57776819,  2.57776819,
        2.57776819,  2.57776819,  2.57776819,  2.53224286,  2.53224286,
        2.53224286,  2.53224286,  2.53224286,  2.53224286,  2.36511138,
        2.40615415,  2.35459012,  2.1623917 ,  2.1623917 ,  2.09